In [ ]:
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.animation import FuncAnimation

# Open simulation netcdf files

In [ ]:
data_dir = "/Users/elischwat/Development/data/sublimationofsnow/lattice_boltzmann_rotors/ethan_output/output_eastriver"

In [ ]:
# List all files in the data directory
files = [f for f in os.listdir(data_dir) if f.endswith('.nc')]

# Function to extract timestep from filename
def extract_timestep(filename):
    # Assuming the timestep is the last 4 characters before the file extension
    return int(filename[-7:-3])

# Dictionary to store data with new time index
data_with_time_index = {}

# Process each file
for file in files:
    # Extract timestep
    timestep = extract_timestep(file)
    
    # Open the file
    file_path = os.path.join(data_dir, file)
    data = xr.open_dataset(file_path)
    
    # Create a new time index based on the timestep
    data = data.assign_coords(time=timestep)
    
    # Store the data
    data_with_time_index[file] = data

# Now data_with_time_index contains all the data with the new time index

In [ ]:
output_dataset = xr.concat(data_with_time_index.values(), dim='time').sortby('time')

## Organize the simulation results

In [ ]:
# Select data where y=1 and y=2 (the y index represents the velocity component)
u_data = output_dataset.sel(y=0).sel(r=0)
v_data = output_dataset.sel(y=1).sel(r=0)

# combine the u and v into one dataset
vel_data = xr.merge([
    u_data.rename_vars(u='u'),
    v_data.rename_vars(u='v')
])

### Open the terrain profile

In [ ]:
profile_ds = xr.open_dataset('profile.nc')
profile_ds.to_dataframe().index.diff().unique()

In [ ]:
# Assign coordinates to transform from simulation coordinates to real space. 
# Ethan extracted the profile dataset from -1500 units to +5000 units, every 3rd cell.
# dx = 1.99993 (from my profile, see above). So to add coordinates to the simulated velocity data, do the following:

dx_profile = 1.99993
dx_simulation = dx_profile * 3

vel_data['x'] = (vel_data.x * dx_simulation).values - 1500*dx_profile
vel_data['z'] = (vel_data.z * dx_simulation).values

## Plot quivers

In [ ]:
X, Z = np.meshgrid(vel_data.x, vel_data.z)

SKIP = 6
plt.figure(figsize=(20,6))
plt.quiver(
    X[::SKIP, ::SKIP],
    Z[::SKIP, ::SKIP],
    vel_data.isel(time=0).u.values[::SKIP, ::SKIP],
    vel_data.isel(time=0).v.values[::SKIP, ::SKIP],
    scale=250,
    width=0.0005
)
plt.fill_between(
    profile_ds.data.to_dataframe().index,
    -20,
    profile_ds.data.to_dataframe().data,
    color='grey'
)
plt.xlim(-2500, 4500)
plt.gca().set_aspect('equal')
plt.show()

# Open real doppler lidar scan, examine data organization

In [ ]:
# Open up an actual scan, mimic plotting from another notebok
actual_scan_file = '/Users/elischwat/Development/data/sublimationofsnow/gucdlrhiM1.b1/gucdlrhiM1.b1.20230419.021833.cdf'
actual_scan_ds = xr.open_dataset(actual_scan_file)
actual_scan_ds['x'] = actual_scan_ds['range']*np.cos(np.deg2rad(actual_scan_ds['elevation']))
actual_scan_ds['z'] = actual_scan_ds['range']*np.sin(np.deg2rad(actual_scan_ds['elevation']))
actual_scan_ds = actual_scan_ds.sel(range=slice(0,4000))
actual_scan_ds = actual_scan_ds.assign(x = - actual_scan_ds.x )

## Plot it. Code from another notebook

In [ ]:
plt.subplots(figsize=(9.6, 4))
plt.contourf(
    actual_scan_ds['x'].values.T, actual_scan_ds['z'].values.T,
    actual_scan_ds['radial_velocity'].values,
    cmap='gist_ncar',
    levels=50
)
plt.colorbar()
plt.gca().set_aspect('equal')
name = actual_scan_file.split('gucdlrhiM1.b1/gucdlrhiM1.b1.')[1][:-4]
plt.title(name)
plt.xlim(-2500,3000)
plt.ylim(0,2500)
plt.show()


## Examine real scan elevation angle values

In [ ]:
np.unique(actual_scan_ds.elevation)

In [ ]:
sorted(np.diff(sorted(np.unique(actual_scan_ds.elevation))))

We can see the the doppler lidar is programed to scan every 1˚ between 0 and 180˚ but it doesn't always have an exact angle, it only varies by ~0.01˚ though. So we set our simulated doppler lidar scan to do that.

In [ ]:
SIMULATED_DOPPLER_LIDAR_ELEVATION_VALUES = np.linspace(0,180, 181)
SIMULATED_DOPPLER_LIDAR_ELEVATION_VALUES

## Examine real scan range values

For simulating the range values, we just take the range values from the actual scan, and use them exactly. The real doppler lidar maximum range is really too high for our simulation, we cut it off at a max of 50000

In [ ]:
np.unique(actual_scan_ds.range)

In [ ]:
SIMULATED_DOPPLER_LIDAR_RANGE_VALUES = np.sort(np.unique(actual_scan_ds.range))
SIMULATED_DOPPLER_LIDAR_RANGE_VALUES = SIMULATED_DOPPLER_LIDAR_RANGE_VALUES[SIMULATED_DOPPLER_LIDAR_RANGE_VALUES < 5000]
SIMULATED_DOPPLER_LIDAR_RANGE_VALUES

## Examine real scan time values

In [ ]:
np.unique(actual_scan_ds.time.diff(dim='time'))/10**9

In [ ]:
(actual_scan_ds.time.max() - actual_scan_ds.time.min()) / 10**9

ok so it takes about 1 second for each range value

# Simulate doppler lidar scan (one frame)

We've adjusted the simulation data into real space, so the doppler lidar is at 0,0.

In [ ]:
vel_data_one_frame = vel_data.isel(time=0)
vel_data_one_frame

In [ ]:
# define function for converting from radial to polar coordinates
# remember: WE GO COUNTERCLOCKWISE FROM 0˚ on the unit circle
def cartesian_convert(elevation, range):
    x = range*np.cos(np.deg2rad(elevation))
    y = range*np.sin(np.deg2rad(elevation))
    return x, y

In [ ]:
time_seconds = 0
df_list = []
# iterate over elevation values and range values
for elevation in SIMULATED_DOPPLER_LIDAR_ELEVATION_VALUES:
    for range in SIMULATED_DOPPLER_LIDAR_RANGE_VALUES:
        x, z = cartesian_convert(elevation, range)
        u = vel_data_one_frame.u.interp(x=x, z=z, method='linear')
        v = vel_data_one_frame.v.interp(x=x, z=z, method='linear')
        radial_velocity_magnitude = u*np.cos(np.deg2rad(elevation)) + v*np.sin(np.deg2rad(elevation))

        df_list.append({
            'u'         : u.values.item(),
            'v'         : v.values.item(),
            'elevation' : elevation,
            'range'     : range,
            'time'      : time_seconds,
            'x'         : x,
            'z'         : z,
            'radial_velocity'     : radial_velocity_magnitude.values.item()
        })
    # iterate 1 second each new elevation angle
    time_seconds += 1

In [ ]:
simulated_scan_df = pd.DataFrame(df_list)

## Plot the simulated scan (scatter plot)

In [ ]:
plt.figure(figsize=(20,6))
plt.scatter(
    simulated_scan_df.x, simulated_scan_df.z, c=simulated_scan_df.radial_velocity,
    cmap='gist_ncar',
    # cmap='RdYlBu',
    vmin=-4,
    vmax=4
)
plt.colorbar(label='radial velocity (m/s)')
plt.fill_between(
    profile_ds.data.to_dataframe().index,
    -20,
    profile_ds.data.to_dataframe().data,
    color='grey'
)
plt.xlim(-2500, 4500)
plt.gca().set_aspect('equal')
plt.show()

## Plot the simulated scan (contourf plot)

We need the actual scan and simulated scan to match, in terms of data structure...

In [ ]:
simulated_scan_ds = simulated_scan_df.set_index(['time', 'range']).to_xarray()
# to match the real scan 
simulated_scan_ds = simulated_scan_ds.sel(range=slice(0,4000))

ACTUAL SCAN PLOTTING CODE

In [ ]:
plt.figure(figsize=(20,6))
plt.contourf(
    actual_scan_ds['x'].values.T, actual_scan_ds['z'].values.T,
    actual_scan_ds['radial_velocity'].values,
    cmap='gist_ncar',
    levels=50
)
plt.colorbar(label='Radial velocity (m/s)')
name = actual_scan_file.split('gucdlrhiM1.b1/gucdlrhiM1.b1.')[1][:-4]
plt.title('Real scan from ' + name)

plt.fill_between(
    profile_ds.data.to_dataframe().index,
    -20,
    profile_ds.data.to_dataframe().data,
    color='grey'
)
plt.xlim(-2500,3000)
plt.ylim(0,2500)
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
plt.figure(figsize=(20,6))
plt.contourf(
    simulated_scan_ds['x'].values.T, simulated_scan_ds['z'].values.T,
    simulated_scan_ds['radial_velocity'].values.T,
    cmap='gist_ncar',
    levels=50,
    vmin=-4,
    vmax=4
)
plt.colorbar(label='Radial velocity (m/s)')
name = actual_scan_file.split('gucdlrhiM1.b1/gucdlrhiM1.b1.')[1][:-4]
plt.title('Simulated scan from idealized simulation')
plt.fill_between(
    profile_ds.data.to_dataframe().index,
    -20,
    profile_ds.data.to_dataframe().data,
    color='grey'
)
plt.xlim(-2500,3000)
plt.ylim(0,2500)
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
X, Z = np.meshgrid(vel_data.x, vel_data.z)

SKIP = 6
plt.figure(figsize=(20,6))
plt.quiver(
    X[::SKIP, ::SKIP],
    Z[::SKIP, ::SKIP],
    vel_data.isel(time=0).u.values[::SKIP, ::SKIP],
    vel_data.isel(time=0).v.values[::SKIP, ::SKIP],
    scale=250,
    width=0.0005
)
plt.fill_between(
    profile_ds.data.to_dataframe().index,
    -20,
    profile_ds.data.to_dataframe().data,
    color='grey'
)
plt.xlim(-2500,3000)
plt.ylim(0,2500)
plt.gca().set_aspect('equal')
plt.show()